# Preprocessing Part 1 - Handling missing values, variable imputation and first cardinality reduction

In [111]:
#Importing all libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from itertools import combinations
from scipy.stats import chi2_contingency
import os

In [112]:
print(os.getcwd())

c:\Users\ntama\Documentos\Data Science\Codecademy\projects\Date-A-Scientist - Final Project\OKCupid-Date-A-Scientist-Starter\date-a-scientist-project\notebooks


Importing the dataset *'profiles.csv'*

In [113]:
profiles = pd.read_csv("../data/profiles_eda.csv")
profiles = profiles.loc[:, ~profiles.columns.str.contains('^Unnamed')]
print(profiles.columns)

Index(['age', 'body_type', 'diet', 'drinks', 'drugs', 'education', 'ethnicity',
       'height', 'income', 'job', 'last_online', 'location', 'offspring',
       'orientation', 'pets', 'religion', 'sex', 'sign', 'smokes', 'speaks',
       'status'],
      dtype='object')


### 1.- *'body_type'*

In [114]:
profiles['body_type'].value_counts()

body_type
average           14652
fit               12711
athletic          11819
thin               4711
curvy              3924
a little extra     2629
skinny             1777
full figured       1009
overweight          444
jacked              421
used up             355
rather not say      198
Name: count, dtype: int64

Reducing from 12 values to 6

In [115]:
#grouping body_type to reduce values
body_type_map = {
    'average': 'regular',
    'fit': 'regular',
    'athletic': 'regular',
    'jacked': 'regular',
    'thin': 'regular',
    'skinny': 'regular',
    'curvy': 'large',
    'full figured': 'large',
    'a little extra': 'large',
    'overweight': 'large',
    'used up': 'large',
    'rather not say': 'unknown'
}

profiles['body_type'] = profiles['body_type'].map(body_type_map)

profiles['body_type'] = profiles['body_type'].fillna('unknown')


In [116]:
print(np.sum(profiles['body_type'] == 'unknown'))

5494


### 2.- *'diet'*

Reducing from 18 values to 6

In [117]:
#grouping diet to reduce values
diet_type_map = {
    'anything': 'anything',
    'mostly anything': 'anything',
    'strictly anything': 'anything',
    'vegetarian': 'vegetarian',
    'mostly vegetarian': 'vegetarian',
    'strictly vegetarian': 'vegetarian',
    'other': 'other',
    'mostly other': 'other',
    'strictly other': 'other',
    'vegan': 'vegan',
    'mostly vegan': 'vegan',
    'strictly vegan': 'vegan',
    'kosher': 'kosher',
    'mostly kosher': 'kosher',
    'strictly kosher': 'kosher',
    'halal': 'halal',
    'mostly halal': 'halal',
    'strictly halal': 'halal',
}

profiles['diet'] = profiles['diet'].map(diet_type_map)

profiles['diet'] = profiles['diet'].fillna('unknown')

In [118]:
print(profiles['diet'].value_counts())

diet
anything      27881
unknown       24395
vegetarian     4986
other          1790
vegan           702
kosher          115
halal            77
Name: count, dtype: int64


### 3.- *'drinks'*

In [119]:
profiles['drinks'].value_counts()

drinks
socially       41780
rarely          5957
often           5164
not at all      3267
very often       471
desperately      322
Name: count, dtype: int64

Reducing from 6 values to 3

In [120]:
#grouping drinks to three values
drink_type_map = {
    'desperately': 'a lot',
    'very often': 'a lot',
    'often': 'a lot',
    'socially': 'a little',
    'rarely': 'a little',
    'not at all': 'not at all'
}

profiles['drinks'] = profiles['drinks'].map(drink_type_map)

profiles['drinks'] = profiles['drinks'].fillna('unknown')

### 4.- *'drugs'*

In [121]:
print(profiles['drugs'].value_counts())

profiles['drugs'] = profiles['drugs'].fillna('unknown')

drugs
never        37724
sometimes     7732
often          410
Name: count, dtype: int64


### 5.- *'education'*

In [122]:
print(profiles['education'].value_counts())

education
graduated from college/university    23959
graduated from masters program        8961
working on college/university         5712
working on masters program            1683
graduated from two-year college       1531
graduated from high school            1428
graduated from ph.d program           1272
graduated from law school             1122
working on two-year college           1074
dropped out of college/university      995
working on ph.d program                983
college/university                     801
graduated from space camp              657
dropped out of space camp              523
graduated from med school              446
working on space camp                  445
working on law school                  269
two-year college                       222
working on med school                  212
dropped out of two-year college        191
dropped out of masters program         140
masters program                        136
dropped out of ph.d program            127
d

In [123]:
#reducing education
def simplify_education(edu):
    if pd.isnull(edu):
        return 'unknown'
    edu = edu.lower()
    if 'ph.d' in edu:
        return 'phd'
    elif 'med school' in edu:
        return 'med'
    elif 'law school' in edu:
        return 'law'
    elif 'masters' in edu:
        return 'masters'
    elif 'college/university' in edu:
        return 'college'
    elif 'two-year college' in edu:
        return 'two_year_college'
    elif 'high school' in edu:
        return 'high_school'
    elif 'space camp' in edu:
        return 'other'
    else:
        return 'other'

profiles['education'] = profiles['education'].apply(simplify_education)

profiles['education'] = profiles['education'].fillna('unknown')


In [124]:
print(profiles['education'].value_counts())

education
college             31467
masters             10920
unknown              6628
two_year_college     3018
phd                  2408
high_school          1713
other                1683
law                  1428
med                   681
Name: count, dtype: int64


### 6.- *'ethnicity'*

From proportions extracted from EDA, MCAR is more likely, since there's no clear reason why the data is missing and all proportions are roughly equal.</br>
Every missing value will be replaced with 'unknown'</br>

In [125]:
print(profiles['ethnicity'].value_counts())
print(profiles['ethnicity'].unique())

ethnicity
white                                                                 32831
asian                                                                  6134
hispanic / latin                                                       2823
black                                                                  2008
other                                                                  1706
                                                                      ...  
middle eastern, indian, white                                             1
asian, middle eastern, black, white, other                                1
asian, middle eastern, indian, hispanic / latin, white, other             1
black, native american, indian, pacific islander, hispanic / latin        1
asian, black, indian                                                      1
Name: count, Length: 217, dtype: int64
['asian, white' 'white' nan 'asian, black, other' 'white, other'
 'hispanic / latin, white' 'hispanic / latin' 'pac

In [126]:
print(profiles['ethnicity'].nunique())

217


It is clear that most of the 217 values are likely rare, appearing only a few times. These rare categories can introduce noise, skew distributions, and make it difficult to draw reliable statistical conclusions. </br>
reducing it is necessary for analytical clarity, statistical validity, and model performance. </br>
Let's group rare ethcnicities (less than 50 entries)

In [127]:
ethnicity_counts = profiles['ethnicity'].value_counts()
rare_ethnicities = ethnicity_counts[ethnicity_counts < 50].index
profiles['ethnicity_grouped'] = profiles['ethnicity'].replace(rare_ethnicities, 'rare_ethnicity')


In [128]:
print(profiles['ethnicity_grouped'].value_counts())

ethnicity_grouped
white                                                                                                      32831
asian                                                                                                       6134
hispanic / latin                                                                                            2823
black                                                                                                       2008
other                                                                                                       1706
hispanic / latin, white                                                                                     1301
rare_ethnicity                                                                                              1147
indian                                                                                                      1077
asian, white                                                                  

### 7.- *'height'*

In [129]:
print(profiles['height'].value_counts(0))

height
70.0    6074
68.0    5449
67.0    5354
72.0    5315
69.0    5179
        ... 
3.0        1
8.0        1
93.0       1
52.0       1
4.0        1
Name: count, Length: 61, dtype: int64


In [130]:
#Binning our data to check the counts in intervals
print('Frequency table for height.')

height_list = list(range(0,101,5))
binned_height = pd.cut(profiles['height'], height_list, right=False)
print(binned_height.value_counts())

Frequency table for height.
height
[65, 70)     24538
[70, 75)     21577
[60, 65)     10757
[75, 80)      2634
[55, 60)       296
[80, 85)        70
[95, 100)       20
[35, 40)        13
[50, 55)        12
[90, 95)         9
[85, 90)         6
[45, 50)         4
[5, 10)          3
[40, 45)         3
[0, 5)           3
[25, 30)         1
[30, 35)         0
[20, 25)         0
[15, 20)         0
[10, 15)         0
Name: count, dtype: int64


Capping implausible outliers: Heights under 58 and over 84 are unlikely to be real.

In [131]:
profiles['height'] = profiles['height'].clip(lower=58, upper=84)

In [132]:
#Binning our data to check the counts in intervals
print('Table 2: Modified frequency table for height.')

height_list = list(range(50,85,5))
binned_height = pd.cut(profiles['height'], height_list, right=False)
print(binned_height.value_counts())

Table 2: Modified frequency table for height.
height
[65, 70)    24538
[70, 75)    21577
[60, 65)    10757
[75, 80)     2634
[55, 60)      335
[50, 55)        0
Name: count, dtype: int64


### 8.- *'income'*

In [133]:
profiles['income'].value_counts(1)

income
-1          0.808094
 20000      0.049244
 100000     0.027041
 80000      0.018533
 30000      0.017482
 40000      0.016765
 50000      0.016265
 60000      0.012278
 70000      0.011794
 150000     0.010526
 1000000    0.008691
 250000     0.002486
 500000     0.000801
Name: proportion, dtype: float64

Feature *'income'* has 81% missing values or reported as "-1". imputation is likely to introduce noise. 

In [134]:
#Removing 'income'
profiles = profiles.drop(['income'], axis=1)

In [135]:
print(profiles.columns)

Index(['age', 'body_type', 'diet', 'drinks', 'drugs', 'education', 'ethnicity',
       'height', 'job', 'last_online', 'location', 'offspring', 'orientation',
       'pets', 'religion', 'sex', 'sign', 'smokes', 'speaks', 'status',
       'ethnicity_grouped'],
      dtype='object')


### 9.- *'job'*

In [136]:
profiles['job'].value_counts()

job
other                                7589
student                              4882
science / tech / engineering         4848
computer / hardware / software       4709
artistic / musical / writer          4439
sales / marketing / biz dev          4391
medicine / health                    3680
education / academia                 3513
executive / management               2373
banking / financial / real estate    2266
entertainment / media                2250
law / legal services                 1381
hospitality / travel                 1364
construction / craftsmanship         1021
clerical / administrative             805
political / government                708
rather not say                        436
transportation                        366
unemployed                            273
retired                               250
military                              204
Name: count, dtype: int64

In [137]:
#simplifying job
def simplify_job(job):
    if pd.isnull(job):
        return 'unknown'
    job = job.lower()
    
    if job in ['science / tech / engineering', 'computer / hardware / software']:
        return 'stem'
    elif job in ['artistic / musical / writer', 'entertainment / media']:
        return 'creative'
    elif job in ['sales / marketing / biz dev', 'executive / management', 'banking / financial / real estate']:
        return 'business'
    elif job == 'education / academia':
        return 'education'
    elif job == 'medicine / health':
        return 'health'
    elif job == 'law / legal services':
        return 'law'
    elif job in ['hospitality / travel', 'clerical / administrative', 'transportation', 'construction / craftsmanship']:
        return 'service'
    elif job == 'student':
        return 'student'
    elif job in ['political / government', 'military']:
        return 'government'
    elif job in ['unemployed', 'retired']:
        return 'not_working'
    else:
        return 'unknown'

profiles['job'] = profiles['job'].apply(simplify_job)

profiles['job'] = profiles['job'].fillna('unknown')


In [138]:
profiles['job'].value_counts()

job
unknown        16223
stem            9557
business        9030
creative        6689
student         4882
health          3680
service         3556
education       3513
law             1381
government       912
not_working      523
Name: count, dtype: int64

### 10.- *'last_online'*

In [139]:
print(profiles['last_online'].head())

0    2012-06-28-20-30
1    2012-06-29-21-41
2    2012-06-27-09-10
3    2012-06-28-14-22
4    2012-06-27-21-26
Name: last_online, dtype: object


Session data won't give the model useful information

In [140]:
#drop last_online
profiles = profiles.drop(['last_online'], axis=1)

### 11.- *'location'*

In [141]:
#reducing location cardinality by keeping what is after the comma
profiles['location'] = profiles['location'].str.split(',').str[1].str.strip()

print(len(profiles['location'].value_counts()))

41


In [142]:
print(profiles['location'].value_counts(1))

location
california              0.998482
new york                0.000284
illinois                0.000133
massachusetts           0.000083
texas                   0.000067
oregon                  0.000067
michigan                0.000067
arizona                 0.000050
florida                 0.000050
district of columbia    0.000033
washington              0.000033
utah                    0.000033
georgia                 0.000033
minnesota               0.000033
colorado                0.000033
united kingdom          0.000033
ohio                    0.000033
spain                   0.000033
virginia                0.000033
hawaii                  0.000033
louisiana               0.000017
tennessee               0.000017
mexico                  0.000017
switzerland             0.000017
netherlands             0.000017
pennsylvania            0.000017
germany                 0.000017
missouri                0.000017
british columbia        0.000017
rhode island            0.000017
c

The dataset is 99,8% people from california, so it won't give much information; therefore it can be dropped

In [143]:
profiles = profiles.drop(['location'], axis=1)

### 12.- *'offspring'*

In [144]:
profiles['offspring'].value_counts()

offspring
doesn&rsquo;t have kids                                7560
doesn&rsquo;t have kids, but might want them           3875
doesn&rsquo;t have kids, but wants them                3565
doesn&rsquo;t want kids                                2927
has kids                                               1883
has a kid                                              1881
doesn&rsquo;t have kids, and doesn&rsquo;t want any    1132
has kids, but doesn&rsquo;t want more                   442
has a kid, but doesn&rsquo;t want more                  275
has a kid, and might want more                          231
wants kids                                              225
might want kids                                         182
has kids, and might want more                           115
has a kid, and wants more                                71
has kids, and wants more                                 21
Name: count, dtype: int64

In [145]:
def simplify_offspring(offs):
    if pd.isnull(offs):
        return 'unknown'
    offs = offs.lower()

    if 'has a kid' in offs or 'has kids' in offs:
        return 'has kids'
    elif 'doesn&rsquo;t have kids' in offs:
        return 'no kids'
    else:
        return 'unsure'
    
profiles['offspring'] = profiles['offspring'].apply(simplify_offspring)

profiles['offspring'] = profiles['offspring'].fillna('unknown')

In [146]:
print(profiles['offspring'].value_counts())

offspring
unknown     35561
no kids     16132
has kids     4919
unsure       3334
Name: count, dtype: int64


### 13.- *'orientation'*

Already processed

In [147]:
print(profiles['orientation'].value_counts())

orientation
straight    51606
gay          5573
bisexual     2767
Name: count, dtype: int64


### 14.- *'pets'*

In [148]:
profiles['pets'].value_counts()

pets
likes dogs and likes cats          14814
likes dogs                          7224
likes dogs and has cats             4313
has dogs                            4134
has dogs and likes cats             2333
likes dogs and dislikes cats        2029
has dogs and has cats               1474
has cats                            1406
likes cats                          1063
has dogs and dislikes cats           552
dislikes dogs and likes cats         240
dislikes dogs and dislikes cats      196
dislikes cats                        122
dislikes dogs and has cats            81
dislikes dogs                         44
Name: count, dtype: int64

In [149]:
#creating two new columns: one for dogs, one for cats

profiles['dogs'] = profiles['pets'].apply(lambda x: 'has dogs' if 'has dogs' in str(x) else ('likes dogs' if 'likes dogs' in str(x) else 'no dogs'))

In [150]:
profiles['cats'] = profiles['pets'].apply(lambda x: 'has cats' if 'has cats' in str(x) else ('likes cats' if 'likes cats' in str(x) else 'no cats'))

In [151]:
profiles[['pets', 'dogs', 'cats']].head()

,pets,dogs,cats
0,likes dogs and likes cats,likes dogs,likes cats
1,likes dogs and likes cats,likes dogs,likes cats
2,has cats,no dogs,has cats
3,likes cats,no dogs,likes cats
4,likes dogs and likes cats,likes dogs,likes cats


In [152]:
#dropping 'pets' and keeping just dogs and cats
profiles = profiles.drop(['pets'], axis=1)

### 15.- *'religion'*

In [153]:
profiles['religion'].value_counts()

religion
agnosticism                                   2724
other                                         2691
agnosticism but not too serious about it      2636
agnosticism and laughing about it             2496
catholicism but not too serious about it      2318
atheism                                       2175
other and laughing about it                   2119
atheism and laughing about it                 2074
christianity                                  1957
christianity but not too serious about it     1952
other but not too serious about it            1554
judaism but not too serious about it          1517
atheism but not too serious about it          1318
catholicism                                   1064
christianity and somewhat serious about it     927
atheism and somewhat serious about it          848
other and somewhat serious about it            846
catholicism and laughing about it              726
judaism and laughing about it                  681
buddhism but not too s

In [154]:
#reudcing religion cardinality by keeping only the first word
profiles['religion'] = profiles['religion'].str.extract(r'^(\w+)')
#filling NaN values
profiles['religion'] = profiles['religion'].fillna('unknown')

### 16.- *'sign'*

In [155]:
profiles['sign'].value_counts()

sign
gemini and it&rsquo;s fun to think about         1782
scorpio and it&rsquo;s fun to think about        1772
leo and it&rsquo;s fun to think about            1692
libra and it&rsquo;s fun to think about          1649
taurus and it&rsquo;s fun to think about         1640
cancer and it&rsquo;s fun to think about         1597
pisces and it&rsquo;s fun to think about         1592
sagittarius and it&rsquo;s fun to think about    1583
virgo and it&rsquo;s fun to think about          1574
aries and it&rsquo;s fun to think about          1573
aquarius and it&rsquo;s fun to think about       1503
virgo but it doesn&rsquo;t matter                1497
leo but it doesn&rsquo;t matter                  1457
cancer but it doesn&rsquo;t matter               1454
gemini but it doesn&rsquo;t matter               1453
taurus but it doesn&rsquo;t matter               1450
libra but it doesn&rsquo;t matter                1408
aquarius but it doesn&rsquo;t matter             1408
capricorn and it&rsquo;

In [156]:
#reducing sign's cardinality by keeping only the first word
profiles['sign'] = profiles['sign'].str.extract(r'^(\w+)')

profiles['sign'] = profiles['sign'].fillna('unknown')

### 17.- *'smokes'*

In [157]:
profiles['smokes'].value_counts()

smokes
no                43896
sometimes          3787
when drinking      3040
yes                2231
trying to quit     1480
Name: count, dtype: int64

In [158]:
#reducing the variable 'smokes' from 5 values to 3: yes, no, not answered
def simplify_smokes(smoke):
    if pd.isnull(smoke):
        return 'not answered'
    smoke = smoke.lower()

    if 'sometimes' in smoke:
        return 'yes'
    elif 'when drinking' in smoke:
        return 'yes'
    elif 'trying to quit' in smoke:
        return 'yes'
    else: 
        return 'no'
    
profiles['smokes'] = profiles['smokes'].apply(simplify_smokes)
profiles['smokes'] = profiles['smokes'].fillna('unknown')
profiles['smokes'].value_counts()


smokes
no              46127
yes              8307
not answered     5512
Name: count, dtype: int64

### 18.- *'speaks'*

In [159]:
profiles['speaks'].value_counts()

speaks
english                                                                21828
english (fluently)                                                      6628
english (fluently), spanish (poorly)                                    2059
english (fluently), spanish (okay)                                      1917
english (fluently), spanish (fluently)                                  1288
                                                                       ...  
english (fluently), urdu (poorly), japanese (poorly), french (okay)        1
english, spanish, hindi, c++                                               1
english (fluently), japanese (okay), thai (okay), chinese (poorly)         1
english (fluently), french (okay), italian (okay), hebrew (okay)           1
english (fluently), french, farsi                                          1
Name: count, Length: 7647, dtype: int64

In [160]:
#The variable 'speaks' introduces noise to the data, let's drop it
profiles = profiles.drop(['speaks'], axis=1)

In [161]:
#turning categorical variables into a list
categ_var = list((profiles.select_dtypes(include=object)))

#checking missing values after tidying and processing the data
np.sum(profiles[categ_var].isna())

print(categ_var)
print(profiles[categ_var].dtypes)


['body_type', 'diet', 'drinks', 'drugs', 'education', 'ethnicity', 'job', 'offspring', 'orientation', 'religion', 'sex', 'sign', 'smokes', 'status', 'ethnicity_grouped', 'dogs', 'cats']
body_type            object
diet                 object
drinks               object
drugs                object
education            object
ethnicity            object
job                  object
offspring            object
orientation          object
religion             object
sex                  object
sign                 object
smokes               object
status               object
ethnicity_grouped    object
dogs                 object
cats                 object
dtype: object


c:\Users\ntama\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


### 19.- *'age'*

In [162]:
profiles['age'] = profiles['age'].clip(upper=70)

## Exporting EDA's dataset

In [163]:
profiles.to_csv('profiles_preprocessed1.csv')

In [164]:
profiles.head()

,age,body_type,diet,drinks,drugs,education,ethnicity,height,job,offspring,orientation,religion,sex,sign,smokes,status,ethnicity_grouped,dogs,cats
0,22,large,anything,a little,never,college,"asian, white",75.0,service,no kids,straight,agnosticism,m,gemini,yes,single,"asian, white",likes dogs,likes cats
1,35,regular,other,a lot,sometimes,other,white,70.0,service,no kids,straight,agnosticism,m,cancer,no,single,white,likes dogs,likes cats
2,38,regular,anything,a little,unknown,masters,NaN,68.0,unknown,unknown,straight,unknown,m,pisces,no,available,NaN,no dogs,has cats
3,23,regular,vegetarian,a little,unknown,college,white,71.0,student,unsure,straight,unknown,m,pisces,no,single,white,no dogs,likes cats
4,29,regular,unknown,a little,never,college,"asian, black, other",66.0,creative,unknown,straight,unknown,m,aquarius,no,single,rare_ethnicity,likes dogs,likes cats
